In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/data_assign_2

/content/drive/MyDrive/data_assign_2


In [ ]:
import tensorflow as tf
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import os

In [ ]:
# Define paths to your dataset
dataset_path = 'hand_written_digits'

# Define the image dimensions
img_width, img_height = 100, 100

In [ ]:
# Split the dataset into training and validation sets
def split_dataset(dataset_path, validation_split=0.2):
    classes = ['0_digits', '1_digits', '2_digits']

    train_data = []
    val_data = []

    for class_name in classes:
        class_path = os.path.join(dataset_path, class_name)
        image_list = os.listdir(class_path)
        train_images, val_images = train_test_split(image_list, test_size=validation_split, random_state=42)
        train_data.extend([(os.path.join(class_path, img), class_name) for img in train_images])
        val_data.extend([(os.path.join(class_path, img), class_name) for img in val_images])

    return train_data, val_data

train_data, val_data = split_dataset(dataset_path)

In [ ]:
# Data augmentation
datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=False,
    vertical_flip=False
)

In [ ]:
# Define the model
model = Sequential()

model.add(Conv2D(32, (5, 5), activation='relu', input_shape=(img_width, img_height, 3), padding='valid'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (5, 5), activation='relu', padding='valid'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Train the model
batch_size = 32
train_generator = datagen.flow_from_dataframe(dataframe=pd.DataFrame(train_data, columns=['filename', 'class']),
                                              x_col='filename',
                                              y_col='class',
                                              target_size=(img_width, img_height),
                                              batch_size=batch_size,
                                              class_mode='categorical')
val_generator = datagen.flow_from_dataframe(dataframe=pd.DataFrame(val_data, columns=['filename', 'class']),
                                            x_col='filename',
                                            y_col='class',
                                            target_size=(img_width, img_height),
                                            batch_size=batch_size,
                                            class_mode='categorical')

Found 3725 validated image filenames belonging to 3 classes.
Found 933 validated image filenames belonging to 3 classes.


In [ ]:
# Train the model
epochs = 10
model.fit(train_generator, validation_data=val_generator, epochs=epochs)

Epoch 1/10
117/117 [==============================] - 2019s 17s/step - loss: 0.3616 - accuracy: 0.8639 - val_loss: 0.1159 - val_accuracy: 0.9657
Epoch 2/10
117/117 [==============================] - 129s 1s/step - loss: 0.1101 - accuracy: 0.9678 - val_loss: 0.0783 - val_accuracy: 0.9775
Epoch 3/10
117/117 [==============================] - 129s 1s/step - loss: 0.0942 - accuracy: 0.9750 - val_loss: 0.0662 - val_accuracy: 0.9829
Epoch 4/10
117/117 [==============================] - 133s 1s/step - loss: 0.0704 - accuracy: 0.9801 - val_loss: 0.0457 - val_accuracy: 0.9839
Epoch 5/10
117/117 [==============================] - 119s 1s/step - loss: 0.0529 - accuracy: 0.9831 - val_loss: 0.0598 - val_accuracy: 0.9871
Epoch 6/10
117/117 [==============================] - 131s 1s/step - loss: 0.0595 - accuracy: 0.9823 - val_loss: 0.0396 - val_accuracy: 0.9904
Epoch 7/10
117/117 [==============================] - 117s 992ms/step - loss: 0.0552 - accuracy: 0.9836 - val_loss: 0.0516 - val_accuracy: 0

In [ ]:
# Evaluate the model
validation_accuracy = model.evaluate(val_generator)[1]
print(f'Validation accuracy: {validation_accuracy}')

30/30 [==============================] - 11s 358ms/step - loss: 0.0250 - accuracy: 0.9893
Validation accuracy: 0.9892818927764893
